In [5]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup, NavigableString
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import json
import os
from dotenv import load_dotenv
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem

from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.common.exceptions import TimeoutException
from seleniumwire.utils import decode as sw_decode

from webdriver_manager.chrome import ChromeDriverManager
from fake_headers import Headers
import time
import numpy as np
import re

load_dotenv()
API_KEY = os.getenv("API_KEY")

In [6]:
def chrome_driver(webdriver_path=None):
    # Generate a fake user agent
    software_names = [SoftwareName.CHROME.value]
    operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]
    user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

    ua = user_agent_rotator.get_random_user_agent()

    browser_options = ChromeOptions()
    browser_options.add_argument("--headless")
    # browser_options.add_argument("--incognito")
    # browser_options.add_argument("--log-level=3")
    # browser_options.add_argument("--disable-gpu")
    # browser_options.add_argument("--disable-extensions")
    # browser_options.add_argument("--disable-notifications")
    # browser_options.add_argument("--disable-popup-blocking")
    browser_options.add_argument(f"user-agent={ua}")

    if webdriver_path is not None:
        driver = webdriver.Chrome(
            service=ChromeService(executable_path=f"{webdriver_path}"),
            options=browser_options
        )
    else:
        driver = webdriver.Chrome(
            service=ChromeService(ChromeDriverManager().install()),
            options=browser_options)

    return driver

def text_to_int(text):
    # Initialize a multiplier
    multiplier = 1
    
    # Check if 'M' is in the text, indicating millions
    if 'na' in text:
        if 'M' in text:
            multiplier = 1000000
            text = text.replace('M (na) follower', '')  # Remove 'M' for conversion
        elif 'K' in text:
            multiplier = 1000
            text = text.replace('K (na) follower', '')
    else:
        if 'M' in text:
            multiplier = 1000000
            text = text.replace('M followers', '')  # Remove 'M' for conversion
        elif 'K' in text:
            multiplier = 1000
            text = text.replace('K followers', '')
    
    # Extract only the numeric part of the text
    
    # Convert the numeric part to an integer and apply the multiplier
    return int(float(text) * multiplier)

def is_social_media_link(url):
    # Regular expression patterns for matching social media URLs
    patterns = {
        'Facebook': r'https?://(www\.)?facebook\.com/[\w.]+/?',
        'X (Twitter)': r'https?://(www\.)?twitter\.com/[\w.]+/?',
        'Instagram': r'https?://(www\.)?instagram\.com/[\w.]+/?',
        'Youtube': r'https?://(www\.)?(youtube\.com|youtu\.be)/[\w.]+/?',
        'TikTok':r'https?://(www\.)?tiktok\.com/@[\w.]+/?'
    }
    # Check each social media pattern to see if it matches the URL
    for platform, pattern in patterns.items():
        if re.match(pattern, url):
            return True, platform
    
    # If no pattern matches, the URL is not recognized as a social media link
    return False, None

In [21]:
url = "https://multi-club-matches.webapi.gc.eflservices.co.uk/v2/teams?competitionID=10,11,12&seasonID=2023"
concatenated_urls = []
teams = []
payload = {}
headers = {
  'authority': 'multi-club-matches.webapi.gc.eflservices.co.uk',
  'accept-language': 'en-US,en;q=0.9,ja-JP;q=0.8,ja;q=0.7',
  'origin': 'https://www.efl.com',
  'referer': 'https://www.efl.com/',
  'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'sec-fetch-dest': 'empty',
  'sec-fetch-mode': 'cors',
  'sec-fetch-site': 'cross-site'
}

response = requests.request("GET", url, headers=headers, data=payload)

if response.status_code == 200:
    data = response.json()
    for i in data['data']:
        teams.append(i['attributes']['teams']['teamName'])
        concatenated_urls.append(i['attributes']['teams']['teamURL'])

# Initialize a list to hold the concatenated URLs


url_to_follow_items = {}

# Print the concatenated URLs
for parent_url,team in zip(concatenated_urls,teams):
    
    # Fetch the webpage
    try:
        driver = chrome_driver()
        driver.get(parent_url)
        html_content = driver.page_source

        # Parse the HTML content
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all 'li' items with the class 'p-follow__item'
        socials = soup.find_all('a')

        # Initialize a list to hold the hrefs for this parent URL
        href_links = []

        # Extract the 'href' attribute from each 'li' item found
        for item in socials:
            href = item.get('href')
            if href:
                is_valid, platform = is_social_media_link(href) 
            if is_valid:
                href_links.append(href)

        
    except Exception:
        print("Error", team)
        continue
    url_to_follow_items[team] = href_links
    print(f"{team}: {href_links}")
# Print the dictionary


Birmingham City: ['https://www.facebook.com/BCFCofficial/', 'https://twitter.com/BCFC/', 'https://www.instagram.com/bcfc/?hl=en', 'https://www.tiktok.com/@bcfc']
Blackburn Rovers: ['https://www.facebook.com/1Rovers', 'https://twitter.com/rovers', 'https://www.instagram.com/rovers/', 'https://www.youtube.com/channel/UCg4185wSpo9swSCSUEYUGTg', 'https://www.tiktok.com/@rovers', 'https://www.facebook.com/1Rovers', 'https://twitter.com/rovers', 'https://www.instagram.com/rovers/', 'https://www.youtube.com/channel/UCg4185wSpo9swSCSUEYUGTg', 'https://www.tiktok.com/@rovers', 'https://www.youtube.com/watch?v=P8ZoF7XXv_o&ab_channel=BlackburnRoversFootballClub', 'https://www.facebook.com/1Rovers', 'https://twitter.com/rovers', 'https://www.instagram.com/rovers/', 'https://www.youtube.com/channel/UCg4185wSpo9swSCSUEYUGTg', 'https://www.tiktok.com/@rovers']
Bristol City: []
Cardiff City: ['https://www.facebook.com/cardiffcityfc', 'https://www.instagram.com/cardiffcityfc/', 'https://twitter.com/car

In [22]:
sorted_teams = dict(sorted(url_to_follow_items.items()))
sorted_teams

{'AFC Wimbledon': ['http://www.facebook.com/afcwimbledon',
  'http://twitter.com/AFCWimbledon',
  'http://www.youtube.com/user/AFCWimbledonTV',
  'http://www.instagram.com/afc_wimbledon/',
  'https://www.tiktok.com/@afcwimbledon_',
  'https://www.facebook.com/afcwimbledon',
  'https://twitter.com/AFCWimbledon',
  'https://www.youtube.com/user/AFCWimbledonTV',
  'https://www.instagram.com/afc_wimbledon/',
  'https://www.tiktok.com/@afcwimbledon_'],
 'Accrington Stanley': ['https://www.facebook.com/ASFCofficial/',
  'https://twitter.com/ASFCofficial',
  'https://www.youtube.com/stanleyofficial',
  'https://www.instagram.com/asfcofficial/'],
 'Barnsley': ['https://www.facebook.com/BarnsleyFC/',
  'https://twitter.com/BarnsleyFC',
  'https://www.youtube.com/user/1BarnsleyFC',
  'https://www.instagram.com/BarnsleyFC',
  'https://www.tiktok.com/@barnsleyfc',
  'https://www.facebook.com/BarnsleyFC/?',
  'https://twitter.com/BarnsleyFC',
  'https://www.youtube.com/channel/UCli5AB-aSOhTxiF0XZL5

In [31]:
ref = pd.read_excel("efl.xlsx")

In [ ]:
for index, row in ref.iterrows():
    temp = sorted_teams[row['Unnamed: 0']]
    columns = ['Instagram', 'Facebook', 'TikTok','X (Twitter)','Youtube']
    for j in columns:
        for i in temp:
            is_valid, platform = is_social_media_link(str(i))
            if is_valid:
                ref.at[index, platform] = i
    # row['Youtube'] = ([i for i in value if "youtube" in str(i)] + [""])[0]
    # row['Instagram'] = ([i for i in value if "instagram" in str(i)] + [""])[0]
    # row['Facebook'] = ([i for i in value if "facebook" in str(i)] + [""])[0]
    # row['TikTok'] = ([i for i in value if "tiktok" in str(i)] + [""])[0]
    # row['X (Twitter)'] = ([i for i in value if "twitter" in str(i)] + [""])[0]


In [30]:
ref.to_excel('efl.xlsx', index=False)

In [32]:
#fix yt links
for index, row in ref.iterrows():
    response = requests.get(row['Youtube'])
    html_content = response.content

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all divs with the class 'p-forge-list-item'
    list_items = soup.find('link', rel='canonical')
    ref.at[index,'Youtube'] = list_items.get('href')

In [33]:
ref

,Unnamed: 0,YouTube (subscribers),Instagram (followers),Facebook (followers),TikTok (followers),X (Twitter) (followers),Unnamed: 6,Instagram,Facebook,TikTok,X (Twitter),Youtube
0,Birmingham City,NaN,NaN,NaN,NaN,445716,NaN,https://www.instagram.com/bcfc/?hl=en,https://www.facebook.com/BCFCofficial/,https://www.tiktok.com/@bcfc,https://twitter.com/BCFC/,https://www.youtube.com/channel/UCW1HMToSBse9J...
1,Blackburn Rovers,NaN,NaN,NaN,NaN,308749,NaN,https://www.instagram.com/rovers/,https://www.facebook.com/1Rovers,https://www.tiktok.com/@rovers,https://twitter.com/rovers,https://www.youtube.com/channel/UCg4185wSpo9sw...
2,Bristol City,NaN,NaN,NaN,NaN,220254,NaN,https://www.instagram.com/bcfc/,https://www.facebook.com/bristolcityfc/,https://www.tiktok.com/@bristolcityfc,https://twitter.com/BristolCity,https://www.youtube.com/channel/UCq_5VYwAoOvaL...
3,Cardiff City,NaN,NaN,NaN,NaN,398762,NaN,https://www.instagram.com/cardiffcityfc/,https://www.facebook.com/cardiffcityfc,https://www.tiktok.com/@cardiffcityfc,https://twitter.com/cardiffcityfc,https://www.youtube.com/channel/UCfBVy8PAMwyNb...
4,Coventry City,NaN,NaN,NaN,NaN,179548,NaN,https://www.instagram.com/coventrycityfcoffici...,https://www.facebook.com/CoventryCityFC/?fref=ts,https://www.tiktok.com/@ccfcofficial?lang=en,https://twitter.com/Coventry_City,https://www.youtube.com/channel/UCch_NWdo3JWKn...
5,Huddersfield Town,NaN,NaN,NaN,NaN,244947,NaN,https://www.instagram.com/htafc/,https://www.facebook.com/htafc/,https://www.tiktok.com/@htafc,https://twitter.com/htafc,https://www.youtube.com/channel/UCPzdw2bYGCq6X...
6,Hull City,NaN,NaN,NaN,NaN,583136,NaN,https://www.instagram.com/hullcity/?hl=en,https://www.facebook.com/hullcity,https://www.tiktok.com/@hullcity?lang=en,https://twitter.com/HullCity,https://www.youtube.com/channel/UC8MRV5E-Bi5qW...
7,Ipswich Town,NaN,NaN,NaN,NaN,245539,NaN,https://www.instagram.com/ipswichtown/,https://www.facebook.com/officialitfc/,https://www.tiktok.com/@ipswichtown?lang=en,https://twitter.com/IpswichTown,https://www.youtube.com/channel/UCjNwxJec96lMW...
8,Leeds United,NaN,NaN,NaN,NaN,1067004,NaN,https://www.instagram.com/leedsunited,https://www.facebook.com/LeedsUnited,https://www.tiktok.com/@leedsunited,https://twitter.com/lufc,https://www.youtube.com/channel/UCyQcJHDN4uYfP...
9,Leicester City,NaN,NaN,NaN,NaN,2729467,NaN,https://www.instagram.com/lcfc/?hl=en,https://www.facebook.com/lcfc,https://www.tiktok.com/@lcfc?lang=en,https://twitter.com/lcfc,https://www.youtube.com/channel/UCBkRQtxofyXr0...


In [34]:
#youtube
ids = ",".join(str(elem.replace("https://www.youtube.com/channel/","")) for elem in ref['Youtube'])
url = "https://www.googleapis.com/youtube/v3/channels"
params = {
    "part": "statistics",
    "id": ids,
    "key": API_KEY
}

response = requests.get(url, params=params)

if response.status_code == 200:
    file = response.json()
    for i in file['items']:
        index_number = ref.index[ref['Youtube'] == ("https://www.youtube.com/channel/" + str(i['id']))].tolist()
        ref.at[index_number[0],'YouTube (subscribers)'] = i['statistics']['subscriberCount']

C:\Users\kvray\AppData\Local\Temp\ipykernel_82600\2428173603.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '296000' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ref.at[index_number[0],'YouTube (subscribers)'] = i['statistics']['subscriberCount']


In [35]:
#Tiktok
# driver.get('https://www.tiktok.com/@atlhawks')
for index, row in ref.iterrows():
    try:
        driver = chrome_driver()
        driver.get(row['TikTok'])
        state_data = json.loads(driver.execute_script("return document.getElementById('__UNIVERSAL_DATA_FOR_REHYDRATION__').textContent"))
        driver.close()
        driver.quit()
        stats_data = state_data['__DEFAULT_SCOPE__']["webapp.user-detail"]["userInfo"]['stats']
        print(row['TikTok'], " ", stats_data['followerCount'])
        ref.at[index, 'TikTok (followers)'] = stats_data['followerCount']
        time.sleep(2)
    except:
        continue

https://www.tiktok.com/@bcfc   180800
https://www.tiktok.com/@rovers   101000
https://www.tiktok.com/@bristolcityfc   296800
https://www.tiktok.com/@cardiffcityfc   10200
https://www.tiktok.com/@ccfcofficial?lang=en   124800
https://www.tiktok.com/@htafc   150700
https://www.tiktok.com/@hullcity?lang=en   468800
https://www.tiktok.com/@ipswichtown?lang=en   161600
https://www.tiktok.com/@leedsunited   1700000
https://www.tiktok.com/@lcfc?lang=en   2800000
https://www.tiktok.com/@boro   154400
https://www.tiktok.com/@millwallfcofficial   119900
https://www.tiktok.com/@norwichcityfc   1100000
https://www.tiktok.com/@plymouthargylefc   95000
https://www.tiktok.com/@pnefc?lang=en   52600
https://www.tiktok.com/@qprfc?lang=en   132800
https://www.tiktok.com/@officialswfc   122600
https://www.tiktok.com/@southamptonfc   957300
https://www.tiktok.com/@sunderlandafc   446300
https://www.tiktok.com/@swansofficial   409200
https://www.tiktok.com/@watfordfcofficial   1400000
https://www.tiktok.co

In [ ]:
#facebook

# for index, row in ref.iterrows():
#     if row['Facebook (followers)'] > 0:
#         print("finished")
#     else:
#         try:
#             browser_options = ChromeOptions() 
#             browser_options.add_argument('--headless')
#             driver = chrome_driver()
#             driver.get(row['Facebook'])

#             wait = WebDriverWait(driver, 10)  # Wait for up to 10 seconds
#             element_present = wait.until(
#                 EC.presence_of_element_located((By.CSS_SELECTOR, f'a[href="{row['Facebook']}/followers/"]'))
#             )

#             # Now execute the script
#             script = f"""
#             var element = document.querySelector('a[href="{row['Facebook']}/followers/"]');
#             return element ? element.textContent : 'Element not found';
#             """
#             followers_text = driver.execute_script(script)
#             followers_text = text_to_int(followers_text)
#             ref.at[index, 'Facebook (followers)'] = followers_text


#             driver.quit()
#         except:
#             continue

finished
finished
finished
finished
finished
finished
finished
finished
finished
finished
finished
finished
finished
finished
finished


In [36]:
#Instagram

# driver.get('https://www.tiktok.com/@atlhawks')
for index,row in ref.iterrows():
    
    driver = chrome_driver()
    driver.get(row['Instagram'])
    time.sleep(5)
    for i in driver.requests:
        if str(i).startswith('https://www.instagram.com/api/v1/users/web_profile_info'):
            data = sw_decode(i.response.body, i.response.headers.get('Content-Encoding', 'identity'))
            data = data.decode("utf-8")
            data = json.loads(data)
            ref.at[index, 'Instagram (followers)'] = data['data']['user']['edge_followed_by']['count']
    driver.close()
    driver.quit()


In [37]:
#Facebook
def chrome_driver1(webdriver_path=None):
    # Generate a fake user agent
    software_names = [SoftwareName.CHROME.value]
    operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]
    user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

    ua = user_agent_rotator.get_random_user_agent()

    browser_options = ChromeOptions()
    browser_options.add_argument("--headless")
    # browser_options.add_argument("--incognito")
    # browser_options.add_argument("--log-level=3")
    # browser_options.add_argument("--disable-gpu")
    # browser_options.add_argument("--disable-extensions")
    # browser_options.add_argument("--disable-notifications")
    # browser_options.add_argument("--disable-popup-blocking")
    # browser_options.add_argument(f"user-agent={ua}")

    if webdriver_path is not None:
        driver = webdriver.Chrome(
            service=ChromeService(executable_path=f"{webdriver_path}"),
            options=browser_options
        )
    else:
        driver = webdriver.Chrome(
            service=ChromeService(ChromeDriverManager().install()),
            options=browser_options)

    return driver
for index, row in ref.iterrows():
    driver = chrome_driver1()
    driver.get(row['Facebook'])
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    script_tags = soup.find_all('script', attrs={'data-content-len': True})

    # Iterate over these script tags
    script_tags = soup.find_all('script')

    # Iterate over these script tags
    for script in script_tags:
        data_content_len = script.get('data-content-len')
        if data_content_len and int(data_content_len) > 20000:
            try:
                # Parse the script tag's content as JSON
                script_json = json.loads(script.text)
                for i in script_json['require'][0][3][0]['__bbox']['require'][7][3][1]['__bbox']['result']['data']['user']['profile_header_renderer']['user']['profile_social_context']['content']:
                    if i['uri'].endswith("/followers/"):
                        nested_content = i['text']['text']
                        ref.at[index, 'Facebook (followers)'] = text_to_int(nested_content)
                break  # Found the target script tag, no need to continue
            except (KeyError, IndexError, json.JSONDecodeError):
                continue
    driver.quit()
# wait = WebDriverWait(driver, 10)  # Wait for up to 10 seconds
# element_present = wait.until(
#     EC.presence_of_element_located((By.CSS_SELECTOR, f'a[href="https://www.facebook.com/Padres/followers/"]'))
# )

# # Now execute the script
# script = """
# var element = document.querySelector('a[href="https://www.facebook.com/Padres/followers/"]');
# return element ? element.textContent : 'Element not found';
# """
# followers_text = driver.execute_script(script)
# followers_text = text_to_int(followers_text)
# print(followers_text)

In [4]:
# def chrome_driver1(webdriver_path=None):
#     # Generate a fake user agent
#     software_names = [SoftwareName.CHROME.value]
#     operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]
#     user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

#     ua = user_agent_rotator.get_random_user_agent()

#     browser_options = ChromeOptions()
#     # browser_options.add_argument("--headless")
#     # browser_options.add_argument("--incognito")
#     # browser_options.add_argument("--log-level=3")
#     # browser_options.add_argument("--disable-gpu")
#     # browser_options.add_argument("--disable-extensions")
#     # browser_options.add_argument("--disable-notifications")
#     # browser_options.add_argument("--disable-popup-blocking")
#     browser_options.add_argument(f"user-agent={ua}")

#     if webdriver_path is not None:
#         driver = webdriver.Chrome(
#             service=ChromeService(executable_path=f"{webdriver_path}"),
#             options=browser_options
#         )
#     else:
#         driver = webdriver.Chrome(
#             service=ChromeService(ChromeDriverManager().install()),
#             options=browser_options)

#     return driver

# driver = chrome_driver1()
# driver.get("https://www.tiktok.com/@ser.geybin")
# time.sleep(1000)
# driver.close()
# driver.quit()


In [38]:
ref.to_excel('efl.xlsx', index=False)

In [39]:
ref

,Unnamed: 0,YouTube (subscribers),Instagram (followers),Facebook (followers),TikTok (followers),X (Twitter) (followers),Unnamed: 6,Instagram,Facebook,TikTok,X (Twitter),Youtube
0,Birmingham City,56700,271679.0,471000.0,180800.0,445716,NaN,https://www.instagram.com/bcfc/?hl=en,https://www.facebook.com/BCFCofficial/,https://www.tiktok.com/@bcfc,https://twitter.com/BCFC/,https://www.youtube.com/channel/UCW1HMToSBse9J...
1,Blackburn Rovers,35800,185437.0,NaN,101000.0,308749,NaN,https://www.instagram.com/rovers/,https://www.facebook.com/1Rovers,https://www.tiktok.com/@rovers,https://twitter.com/rovers,https://www.youtube.com/channel/UCg4185wSpo9sw...
2,Bristol City,38300,271679.0,228000.0,296800.0,220254,NaN,https://www.instagram.com/bcfc/,https://www.facebook.com/bristolcityfc/,https://www.tiktok.com/@bristolcityfc,https://twitter.com/BristolCity,https://www.youtube.com/channel/UCq_5VYwAoOvaL...
3,Cardiff City,36200,201570.0,564000.0,10200.0,398762,NaN,https://www.instagram.com/cardiffcityfc/,https://www.facebook.com/cardiffcityfc,https://www.tiktok.com/@cardiffcityfc,https://twitter.com/cardiffcityfc,https://www.youtube.com/channel/UCfBVy8PAMwyNb...
4,Coventry City,29000,96220.0,174000.0,124800.0,179548,NaN,https://www.instagram.com/coventrycityfcoffici...,https://www.facebook.com/CoventryCityFC/?fref=ts,https://www.tiktok.com/@ccfcofficial?lang=en,https://twitter.com/Coventry_City,https://www.youtube.com/channel/UCch_NWdo3JWKn...
5,Huddersfield Town,38700,264867.0,NaN,150700.0,244947,NaN,https://www.instagram.com/htafc/,https://www.facebook.com/htafc/,https://www.tiktok.com/@htafc,https://twitter.com/htafc,https://www.youtube.com/channel/UCPzdw2bYGCq6X...
6,Hull City,83900,621588.0,1000000.0,468800.0,583136,NaN,https://www.instagram.com/hullcity/?hl=en,https://www.facebook.com/hullcity,https://www.tiktok.com/@hullcity?lang=en,https://twitter.com/HullCity,https://www.youtube.com/channel/UC8MRV5E-Bi5qW...
7,Ipswich Town,106000,294811.0,NaN,161600.0,245539,NaN,https://www.instagram.com/ipswichtown/,https://www.facebook.com/officialitfc/,https://www.tiktok.com/@ipswichtown?lang=en,https://twitter.com/IpswichTown,https://www.youtube.com/channel/UCjNwxJec96lMW...
8,Leeds United,419000,1198472.0,1400000.0,1700000.0,1067004,NaN,https://www.instagram.com/leedsunited,https://www.facebook.com/LeedsUnited,https://www.tiktok.com/@leedsunited,https://twitter.com/lufc,https://www.youtube.com/channel/UCyQcJHDN4uYfP...
9,Leicester City,538000,7882781.0,8700000.0,2800000.0,2729467,NaN,https://www.instagram.com/lcfc/?hl=en,https://www.facebook.com/lcfc,https://www.tiktok.com/@lcfc?lang=en,https://twitter.com/lcfc,https://www.youtube.com/channel/UCBkRQtxofyXr0...
